# BAN5600 Advanced Big Data Computing & Programming Final Project

NFL Big Data Bowl 2021-Evaluation on passing plays of the defensive team

Team member: Yi-Hsuan Tsai, Yuxiu Wang

Instructor: Dr. Hamidreza Ahady Dolatsara

Data: https://www.kaggle.com/c/nfl-big-data-bowl-2021/data

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'
from matplotlib.pyplot import figure

In [0]:
sp_play = spark.read.csv('/FileStore/tables/plays.csv',inferSchema=True,header=True)
sp_week1 = spark.read.csv('/FileStore/tables/week1.csv',inferSchema=True,header=True)

# 1. Data Preparation

## 1.1 Exclude football data
Since the missing value is from football, which will not be used in our clustering model. Therefore, we filter it out directly.

In [0]:
#sp_week1 = sp_week1.filter(sp_week1.event != 'Football')

In [0]:
sp_week1.show()

+--------------------+-----+-----+----+----+----+------+------+-----+-------+---------------+------------+--------+-------+--------+----------+------+-------------+-----+
 time| x| y| s| a| dis| o| dir|event| nflId| displayName|jerseyNumber|position|frameId| team| gameId|playId|playDirection|route|
+--------------------+-----+-----+----+----+----+------+------+-----+-------+---------------+------------+--------+-------+--------+----------+------+-------------+-----+
2018-09-07 01:07:...|91.73|26.67| 0.0|0.01|0.02|289.57|240.93| None| 310| Matt Ryan| 2| QB| 1| away|2018090600| 75| left| null|
2018-09-07 01:07:...|88.89|36.47|0.01|0.01|0.01|105.63| 66.66| None| 79848|Malcolm Jenkins| 27| SS| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|91.35|44.16|0.02|0.03|0.01|290.45| 16.86| None|2495454| Julio Jones| 11| WR| 1| away|2018090600| 75| left|HITCH|
2018-09-07 01:07:...|86.31|22.01|0.09|0.42|0.01| 70.12|168.91| None|2495613| Corey Graham| 24| FS| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|90.78|36.15| 0.0| 0.0| 0.0|257.61|193.97| None|2533040| Mohamed Sanu| 12| WR| 1| away|2018090600| 75| left|HITCH|
2018-09-07 01:07:...|73.64| 28.7|0.03|0.02|0.01|103.05|219.41| None|2534832| Rodney McLeod| 23| FS| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|98.25|26.63|0.01|0.01| 0.0|265.88|194.51| None|2543583|Devonta Freeman| 24| RB| 1| away|2018090600| 75| left| null|
2018-09-07 01:07:...|86.48|31.12|0.09|0.03|0.04| 95.9| 33.36| None|2552315| Jordan Hicks| 58| MLB| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|82.67|20.53|0.03|0.07|0.01| 81.14|174.57| None|2552689| Ronald Darby| 21| CB| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...| 84.0|43.49|0.14|0.05|0.01|108.23|110.32| None|2555383| Jalen Mills| 31| CB| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|91.29|21.83|0.01|0.02|0.01|250.34|151.83| None|2555415| Austin Hooper| 81| TE| 1| away|2018090600| 75| left| OUT|
2018-09-07 01:07:...|85.63|26.59|0.03|0.04|0.02| 87.69| 38.8| None|2558175| Nate Gerry| 47| LB| 1| home|2018090600| 75| left| null|
2018-09-07 01:07:...|95.13|26.71|0.01|0.01|0.01| 250.1|237.27| None|2559033| Ricky Ortiz| 30| FB| 1| away|2018090600| 75| left| FLAT|
2018-09-07 01:07:...|90.11|26.85| 0.0| 0.0| 0.0| null| null| None| null| Football| null| null| 1|football|2018090600| 75| left| null|
2018-09-07 01:07:...|91.73|26.67| 0.0| 0.0| 0.0|289.57|239.89| None| 310| Matt Ryan| 2| QB| 2| away|2018090600| 75| left| null|
2018-09-07 01:07:...|88.89|36.47|0.01|0.01| 0.0|105.63| 68.49| None| 79848|Malcolm Jenkins| 27| SS| 2| home|2018090600| 75| left| null|
2018-09-07 01:07:...|91.37|44.17|0.03|0.03|0.02|290.45| 29.61| None|2495454| Julio Jones| 11| WR| 2| away|2018090600| 75| left|HITCH|
2018-09-07 01:07:...|86.32| 22.0| 0.1|0.58|0.01| 70.88|119.04| None|2495613| Corey Graham| 24| FS| 2| home|2018090600| 75| left| null|
2018-09-07 01:07:...|90.78|36.15| 0.0| 0.0| 0.0|257.61|190.84| None|2533040| Mohamed Sanu| 12| WR| 2| away|2018090600| 75| left|HITCH|
2018-09-07 01:07:...|73.64| 28.7|0.03|0.02|0.01|104.57|228.17| None|2534832| Rodney McLeod| 23| FS| 2| home|2018090600| 75| left| null|
+--------------------+-----+-----+----+----+----+------+------+-----+-------+---------------+------------+--------+-------+--------+----------+------+-------------+-----+
only showing top 20 rows

In [0]:
sp_week1.printSchema()

root
-- time: timestamp (nullable = true)
-- x: double (nullable = true)
-- y: double (nullable = true)
-- s: double (nullable = true)
-- a: double (nullable = true)
-- dis: double (nullable = true)
-- o: double (nullable = true)
-- dir: double (nullable = true)
-- event: string (nullable = true)
-- nflId: integer (nullable = true)
-- displayName: string (nullable = true)
-- jerseyNumber: integer (nullable = true)
-- position: string (nullable = true)
-- frameId: integer (nullable = true)
-- team: string (nullable = true)
-- gameId: integer (nullable = true)
-- playId: integer (nullable = true)
-- playDirection: string (nullable = true)
-- route: string (nullable = true)

# 2. Feature Engineering

### 2.1 "Event" Variable
First, we reorganize the **event** variable. Since the event dataset have lots of "None". Also, the movement of defensive player could change drastically based on different time. For example, if Cornerbacks change his orientation when the ball is thrown, he is more likely to perform man to man coverage since he has to track his targeting player. One the contrary, if a Cornerback is still facing the line of scrimmage when the ball is thrown, he is more likely to be playing zone coverage. Because of this reason, we reorganize the ‘event’ categorical variable into three categories.

In [0]:
df_week1 = sp_week1.select('*').toPandas()
week_array = np.array(df_week1)
previousEvent = 'ball_snap'
for i, instance in enumerate(week_array):
    event = instance[8]
    frameId = instance[13]
    if (previousEvent == 'ball_snap' and event != 'ball_snap') or frameId == 1:
        week_array[i][8] = 'ball_snap'
        previousEvent = 'ball_snap'
    elif (event == 'ball_snap'):
        previousEvent = 'between_snap'
    elif (previousEvent == 'between_snap' and event != 'pass_forward'):
        week_array[i][8] = 'between_snap'
        previousEvent = 'between_snap'
    elif (event == 'pass_forward'):
        week_array[i][8] = 'after_thrown'
        previousEvent = 'after_thrown'
    elif (previousEvent == 'after_thrown' and frameId != 1):
        week_array[i][8] = 'after_thrown'
        previousEvent = 'after_thrown'
        
new_week1 = pd.DataFrame(week_array, columns=df_week1.columns)
df_week1['event'] = new_week1['event']
new_week1 = df_week1

## 2.2 Creating new features

To create good features that can successfully distinguish man and zone coverages, we refer to (Rishav Dutta, Ronald Yurko, Samuel L. Ventura. "Unsupervised Methods for Identifying Pass Coverage Among Defensive Backs with NFL Player Tracking Data. Apr, 2020)". In this paper, th authors generate a rich set of features describing the coverage types. Therefore, we refer to these features and input some of the features that are influential of distinguishing coverage types into our model. In addition to the initial features that generated by the authors, we also creat three new features to see our feature can bring contribution to this clustering method.

**(1) Features that we refer to**

x_var: Variance in the x coordinate

y_var: Variance in the y coordinate

s_var: Variance in the speed

opp_var: Variance in the distance from the nearest opposing player

mate_var: Variance in the distance from the nearest teammate

opp_mean: Mean distance from the nearest opposing player in each frame

mate_mean: Mean distance from the nearest teammate in each frame

opp_dir_var: Variance in the difference in degrees of the direction of motion between the player and the nearest opposing player

opp_dir_mean: Mean difference in degrees of the direction of motion between the player and the nearest opposing player

rat_mean: Mean ratio of the distance to the nearest opposing player and the distance from the nearest opposing player to the nearest teammate

rat_var: Variance of the ratio of the distance to the nearest opposing player and the distance from the nearest opposing player to the nearest teammate

**(2) Features that generated by us**

One important thing in this year’s tracking dataset is that the NFL provide a new variable this year - player’s orientation. We think this new variable could be very helpful for the clustering model to identify pass coverage type. **Therefore, we generate three new feature for our clustering model.**

opp_ori_var: Variance in the orientation between player and the nearest opposing player

opp_ori_mean: Mean difference in orientation between player and his nearest oppsing player

face_LOS: Does the Cornerback is facing on the line of scrimmage; 1 means "Yes", 0 means "No"

Find the nearest oppnent of the cornerback, defining their ID, location (x, y), direction, orientation, and calculating the nearest distance between Cornerbak and his nearest opponents.

In [0]:
new_week2 = new_week1.groupby(['gameId', 'playId', 'frameId'])
player_xy = {}
for name, new in new_week2:
    player_xy[name] = []
    for row in new.iterrows():
        d = [row[1]['nflId'], row[1]['team'], row[1]['x'], row[1]['y'], row[1]['dir'], row[1]['o']]
        player_xy[name].append(d)

In [0]:
features = list(new_week1.columns)
week1Array = np.array(new_week1)
Near_opp_dist = []
for player in week1Array:
    if player[features.index('team')] != 'football':
        opponentLocations = player_xy[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
        distances = []
        directions = []
        orientations =[]
        opponents = []
        xs = []
        ys = []
        for oppLoca in opponentLocations: 
            if player[features.index('team')] != oppLoca[1] and player[features.index('team')] != 'football' and oppLoca[1] != 'football':
                dx = (player[features.index('x')] - oppLoca[2])**2
                dy = (player[features.index('y')] - oppLoca[3])**2
                dist = np.sqrt(dx+dy)
                distances.append(dist)
                directions.append(oppLoca[4])
                orientations.append(oppLoca[5]) ##
                opponents.append(oppLoca[0])
                xs.append(oppLoca[2])
                ys.append(oppLoca[3])
        minDist = min(distances)
        closestOpponent = opponents[np.argmin(distances)]
        opponentDir = directions[np.argmin(distances)]
        opponentOri = orientations[np.argmin(distances)]
        opponentX = xs[np.argmin(distances)]
        opponentY = ys[np.argmin(distances)]
        summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestOpponent, opponentDir, opponentOri, opponentX, opponentY]
        Near_opp_dist.append(summary)
        
Near_opp_dist = pd.DataFrame(Near_opp_dist, columns=['gameId', 'playId', 'frameId', 'nflId', 'oppMinDist', 'closestOpp(nflId)', 'oppDir', 'oppOri', 'oppX', 'oppY'])
new_week1 = pd.merge(new_week1, Near_opp_dist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])

Find the nearest teammate of the cornerback, defining their ID, location (x, y), direction, orientation, and calculating the nearest distance between Cornerbak and his nearest teammate.

In [0]:
features = list(new_week1.columns)
week1Array = np.array(new_week1)
Near_mate_dist = []
for player in week1Array:
    if player[features.index('team')] != 'football':
        mateLocations = player_xy[(player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')])]
        distances = []
        mates = []
        xs = []
        ys = []
        for mateLoca in mateLocations: 
            if player[features.index('team')] == mateLoca[1] and player[features.index('nflId')] != mateLoca[0] and player[features.index('team')] != 'football' and mateLoca[1] != 'football':
                dx = (player[features.index('x')] - mateLoca[2])**2
                dy = (player[features.index('y')] - mateLoca[3])**2
                dist = np.sqrt(dx+dy)
                distances.append(dist)
                mates.append(mateLoca[0])
                xs.append(oppLoca[2])
                ys.append(oppLoca[3])
        minDist = min(distances)
        closestMate = mates[np.argmin(distances)]
        mateX = xs[np.argmin(distances)]
        mateY = ys[np.argmin(distances)]
        summary = [player[features.index('gameId')], player[features.index('playId')], player[features.index('frameId')], player[features.index('nflId')], minDist, closestMate, mateX, mateY]
        Near_mate_dist.append(summary)

Near_mate_dist = pd.DataFrame(Near_mate_dist, columns=['gameId', 'playId', 'frameId', 'nflId', 'mateMinDist', 'closestMate(nflId)', 'mateX', 'mateY'])
new_week1 = pd.merge(new_week1, Near_mate_dist, how='left', on=['gameId', 'frameId', 'playId', 'nflId'])


## 2.2.1 Feature that we refer to

## (1) x_var, y_var, s_var
x_var: Variance in the x coordinate

y_var: Variance in the y coordinate

s_var: Variance in the speed

In [0]:
#x_var feature baesd on the three time period
x_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['x'].agg(['var']).reset_index().rename(columns={"var": "x_var"})
#y_var feature baesd on the three time period
y_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['y'].agg(['var']).reset_index().rename(columns={"var": "y_var"})
#s_var feature
s_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['s'].agg(['var']).reset_index().rename(columns={"var": "s_var"})

## (2) opp_var & mate_var
opp_var: Variance in the distance from the nearest opposing player
mate_var: Variance in the distance from the nearest teammate

In [0]:
opp_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMinDist'].agg(['var']).reset_index().rename(columns={"var": "opp_var"})
mate_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['mateMinDist'].agg(['var']).reset_index().rename(columns={"var": "mate_var"})

## (3) opp_mean & mate_mean
opp_mean: Mean distance from the nearest opposing player

mate_mean: Mean distance from the nearest teammate

In [0]:
opp_mean = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['oppMinDist'].agg(['mean']).reset_index().rename(columns={"mean": "opp_mean"})
mate_mean = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['mateMinDist'].agg(['mean']).reset_index().rename(columns={"mean": "mate_mean"})

## (4) rat_mean ＆ rat_var
rat_mean: Mean ratio of the distance to the nearest opposing player and the distance from the nearest opposing player to the nearest teammate

rat_var: Variance of the ratio of the distance to the nearest opposing player and the distance from the nearest opposing player to the nearest teammate

In [0]:
ratio = new_week1['oppMinDist'] / np.sqrt((new_week1['oppX'] - new_week1['mateX'])**2 + (new_week1['oppY'] - new_week1['mateY'])**2)
new_week1['opp_mate_dist_ratio'] = ratio
rat_mean = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['opp_mate_dist_ratio'].agg(['mean']).reset_index().rename(columns={"mean": "rat_mean"})
rat_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['opp_mate_dist_ratio'].agg(['var']).reset_index().rename(columns={"var": "rat_var"})

## (5) opp_dir_var & opp_dir_mean
opp_dir_var: Variance difference in degrees of the direction of motion between the player and the nearest opposing player

opp_dir_mean: Mean difference in degrees of the direction of motion between the player and the nearest opposing player

In [0]:
diff_dir = np.absolute(new_week1['dir'] - new_week1['oppDir'])
new_week1['diff_dir'] = diff_dir
opp_dir_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['diff_dir'].agg(['var']).reset_index().rename(columns={"var": "opp_dir_var"})
opp_dir_mean = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['diff_dir'].agg(['mean']).reset_index().rename(columns={"mean": "opp_dir_mean"})

## 2.2.2 Features that generated by us

## (6) opp_ori_var & opp_ori_mean
opp_ori_var: Variance difference in the orientation of motion between the player and the nearest opposing player

opp_ori_mean: Mean difference in the orientation of motion between the player and the nearest opposing player

In [0]:
diff_oir = np.absolute(new_week1['o'] - new_week1['oppOri'])
new_week1['diff_oir'] = diff_oir
opp_ori_var = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['diff_oir'].agg(['var']).reset_index().rename(columns={"var": "opp_ori_var"})
opp_ori_mean = new_week1.groupby(['gameId', 'playId', 'event', 'nflId'])['diff_oir'].agg(['mean']).reset_index().rename(columns={"mean": "opp_ori_mean"})

## (7) face_LOS
face_Los: Does the Cornerback face on the line of scrimmage; 1 means "Yes", 0 means "No"

In [0]:
new_week1["face_LOS"] = np.nan
lst = [new_week1]

for col in lst:
    col.loc[col["o"] < 70, "face_LOS"] = 0
    col.loc[(col["o"] >= 70.0) & (col["o"] <= 110), "face_LOS"] = 1
    col.loc[(col["o"] > 110) & (col["o"] < 250.0), "face_LOS"] = 0
    col.loc[(col["o"] >= 250.0) & (col["o"] <= 290.0), "face_LOS"] = 1
    col.loc[col["o"] >290, "face_LOS"] = 0

## 2.2.3 Combine all new feature into the original data

In [0]:
features = [x_var, y_var, s_var, opp_var, opp_mean, mate_var, mate_mean, opp_dir_var, opp_dir_mean, rat_mean, rat_var, opp_ori_var, opp_ori_mean]
for feature in features:
    new_week1 = pd.merge(new_week1, feature, how='left', on=['gameId', 'event', 'playId', 'nflId'])

# 3. Input features into clustering model
To see if our new features are helpful for the clustering model. We first only input the initial 11 feature into GaussianMixture model. After that, we then combine the initial 11 feature with the 3 features that we generated into GaussianMixture model. Finally, we will compare their performance by silhouettes score.

## 3.1 Set the train dataset with the initial 11 features.

In [0]:
#Choose the data only with cornerbacks
features=['x_var','y_var', 's_var', 'opp_var', 'opp_mean', 'mate_var', 'mate_mean', 'opp_dir_var', 'opp_dir_mean', 'rat_mean', 'rat_var']
only_CB = new_week1.loc[new_week1['position'] == 'CB']
only_CB = only_CB[features].dropna()
only_CB1 = only_CB.drop_duplicates()

sp_only_CB = spark.createDataFrame(only_CB)
sp_only_CB1 = spark.createDataFrame(only_CB1)

In [0]:
#Prepare for machine learning
from pyspark.ml.feature import (VectorAssembler,VectorIndexer)
seed_data = VectorAssembler(inputCols = sp_only_CB1.columns, outputCol='features').transform(sp_only_CB1)

#Scale the data
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

seed_data_scaled = scaler.fit(seed_data).transform(seed_data) #New dataset

In [0]:
#Use GaussianMixture model
from pyspark.ml.clustering import GaussianMixture

gmm = GaussianMixture(featuresCol='scaledFeatures').setK(2).setSeed(101)
model = gmm.fit(seed_data_scaled)

#The result
prediction = model.transform(seed_data_scaled)
#prediction.show()

#Cluster evaluation
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette = ClusteringEvaluator().evaluate(prediction)
print("Silhouette value is = " + str(silhouette))

Silhouette value is = 0.09863687740712329

In [0]:
silhouette_scores = [] 

for k in range(2, 8):
  gmm2 = GaussianMixture(featuresCol='scaledFeatures').setK(k).setSeed(101)
  models = gmm2.fit(seed_data_scaled)
  predictions = models.transform(seed_data_scaled)
  silhouettes = ClusteringEvaluator().evaluate(predictions)
  print("Silhouette value is = " + str(silhouettes))
# Divide data into 2 cluster is the best choice

Silhouette value is = 0.09863687740712329
Silhouette value is = -0.5781261791944613
Silhouette value is = 0.0571980878441311
Silhouette value is = -0.41543038805275967
Silhouette value is = -0.4723674262342532
Silhouette value is = -0.19990634946644362

## The clustering model with the 11 features is only 0.0986

## 3.2 Set the train dataset with the initial 11 features and the 3 features that we generated.

In [0]:
features=['x_var','y_var', 's_var', 'opp_var', 'opp_mean', 'mate_var', 'mate_mean', 'opp_dir_var', 'opp_dir_mean', 'rat_mean', 'rat_var', 'opp_ori_var', 'opp_ori_mean']
only_CB = new_week1.loc[new_week1['position'] == 'CB']
only_CB = only_CB[features].dropna()
only_CB1 = only_CB.drop_duplicates()

sp_only_CB = spark.createDataFrame(only_CB)
sp_only_CB1 = spark.createDataFrame(only_CB1)

In [0]:
#Prepare for machine learning
from pyspark.ml.feature import (VectorAssembler,VectorIndexer)
seed_data = VectorAssembler(inputCols = sp_only_CB1.columns, outputCol='features').transform(sp_only_CB1)

In [0]:
#Scale the data
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [0]:
seed_data_scaled = scaler.fit(seed_data).transform(seed_data) #New dataset

In [0]:
#Use GaussianMixture model
from pyspark.ml.clustering import GaussianMixture

gmm = GaussianMixture(featuresCol='scaledFeatures').setK(2).setSeed(101)
model = gmm.fit(seed_data_scaled)

#The result
prediction = model.transform(seed_data_scaled)
#prediction.show()

#Cluster evaluation
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette = ClusteringEvaluator().evaluate(prediction)
print("Silhouette value is = " + str(silhouette))

Silhouette value is = 0.5759062572198097

In [0]:
silhouette_scores = [] 

for k in range(2, 8):
  gmm2 = GaussianMixture(featuresCol='scaledFeatures').setK(k).setSeed(101)
  models = gmm2.fit(seed_data_scaled)
  predictions = models.transform(seed_data_scaled)
  silhouettes = ClusteringEvaluator().evaluate(predictions)
  print("Silhouette value is = " + str(silhouettes))
# Divide data into 2 cluster is the best choice

Silhouette value is = 0.5759062572198097
Silhouette value is = 0.1692867519845718
Silhouette value is = 0.2723494994439825
Silhouette value is = 0.07351244170289134
Silhouette value is = -0.29734491797791035
Silhouette value is = -0.23351205418097748

## We can see that the Silhouette score increase form 0.0986 to 0.5759, which means the new features that we generated improve the model ability of labeling man coverage and zone coverage.

## 3.4 Fit the scaled features into K-means Cluster

In [0]:
from pyspark.ml.clustering import KMeans
for k in range(2, 5):
  kmeans = KMeans(featuresCol='scaledFeatures',k=k).setSeed(101)
  k_model = kmeans.fit(seed_data_scaled)
  k_predictions = k_model.transform(seed_data_scaled)
  k_silhouette = ClusteringEvaluator().evaluate(k_predictions)
  print("Silhouette value is = " + str(k_silhouette))

Silhouette value is = 0.32579804395148554
Silhouette value is = 0.07599335386968319
Silhouette value is = 0.15106670127784902

## 3.5 Fit the scaled features into BisectingKMeans Cluster

In [0]:
from pyspark.ml.clustering import BisectingKMeans
for k in range(2, 5):
  bkm = BisectingKMeans(featuresCol='scaledFeatures').setK(k).setSeed(101)
  bkm_model = bkm.fit(seed_data_scaled)
  bkm_predictions = bkm_model.transform(seed_data_scaled)
  bkm_silhouette = ClusteringEvaluator().evaluate(bkm_predictions)
  print("Silhouette with squared euclidean distance = " + str(bkm_silhouette))

Silhouette with squared euclidean distance = 0.4133269835245622
Silhouette with squared euclidean distance = 0.11715691683133247
Silhouette with squared euclidean distance = 0.14780248000337667

## Gaussian mixture model with k=2 has the best performance

# 4. Merge the predicting result into our orginal data

In [0]:
#Merge the reasult data into the original data
sp_new_week1 = spark.createDataFrame(new_week1)
new_df = sp_new_week1.join(prediction, on=['x_var','y_var', 's_var', 'opp_var', 'opp_mean', 'mate_var', 'mate_mean', 'opp_dir_var', 'opp_dir_mean', 'rat_mean', 'rat_var', 'opp_ori_var', 'opp_ori_mean'], how='left_outer')

+-----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----------------+------------------+------------------+--------------------+------------------+-----------------+--------------------+-----+-----+----+----+----+------+------+------------+---------+-----------+------------+--------+-------+----+----------+------+-------------+-----+------------------+-----------------+------+------+-----+-----+------------------+------------------+-----+-----+-------------------+--------------------+------------------+--------+--------------------+--------------------+--------------------+----------+
 x_var| y_var| s_var| opp_var| opp_mean| mate_var| mate_mean| opp_dir_var| opp_dir_mean| rat_mean| rat_var| opp_ori_var| opp_ori_mean| time| x| y| s| a| dis| o| dir| event| nflId|displayName|jerseyNumber|position|frameId|team| gameId|playId|playDirection|route| oppMinDist|closestOpp(nflId)|oppDir|oppOri| oppX| oppY| mateMinDist|closestMate(nflId)|mateX|mateY|opp_mate_dist_ratio| diff_dir| diff_oir|face_LOS| features| scaledFeatures| probability|prediction|
+-----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+----------------+------------------+------------------+--------------------+------------------+-----------------+--------------------+-----+-----+----+----+----+------+------+------------+---------+-----------+------------+--------+-------+----+----------+------+-------------+-----+------------------+-----------------+------+------+-----+-----+------------------+------------------+-----+-----+-------------------+--------------------+------------------+--------+--------------------+--------------------+--------------------+----------+
1.376017948717959|0.5328532763532674|1.142254131054131|17.603134940129568|11.283478074825176|7.465888749197344|7.9678237014764575|7775.00150968661|105.01407407407405|0.4985378272134604|0.022930517693807093|2105.4396225071223|87.89925925925924|2018-09-07 01:10:...|43.97|37.24|0.59| 0.7|0.06|119.84| 93.76|between_snap|2555383.0|Jalen Mills| 31.0| CB| 12|home|2018090600| 190| left| null| 7.125735049803634| 2555415.0| 85.05|247.87|50.41|34.19| 5.050831614694752| 2534832.0|33.29|43.92|0.36186291634662715| 8.710000000000008| 128.03| 0.0|[1.37601794871795...|[0.06085181731908...|[0.00112879222228...| 1|
1.376017948717959|0.5328532763532674|1.142254131054131|17.603134940129568|11.283478074825176|7.465888749197344|7.9678237014764575|7775.00150968661|105.01407407407405|0.4985378272134604|0.022930517693807093|2105.4396225071223|87.89925925925924|2018-09-07 01:10:...|44.03|37.24|0.57|0.73|0.06|118.58| 84.98|between_snap|2555383.0|Jalen Mills| 31.0| CB| 13|home|2018090600| 190| left| null| 7.085308744155047| 2555415.0| 64.51|257.15|50.43| 34.2| 5.061867244406948| 2534832.0|33.29|43.92| 0.3595825210549776| 20.47| 138.57| 0.0|[1.37601794871795...|[0.06085181731908...|[0.00112879222228...| 1|
1.376017948717959|0.5328532763532674|1.142254131054131|17.603134940129568|11.283478074825176|7.465888749197344|7.9678237014764575|7775.00150968661|105.01407407407405|0.4985378272134604|0.022930517693807093|2105.4396225071223|87.89925925925924|2018-09-07 01:10:...|44.08|37.25|0.59|0.97|0.06|117.88| 70.44|between_snap|2555383.0|Jalen Mills| 31.0| CB| 14|home|2018090600| 190| left| null| 7.076750666796167| 2555415.0| 59.13|275.45|50.48|34.23| 5.102979521808802| 2534832.0|33.29|43.92|0.35862475997609217| 11.309999999999995| 157.57| 0.0|[1.37601794871795...|[0.06085181731908...|[0.00112879222228...| 1|
1.376017948717959|0.5328532763532674|1.142254131054131|17.603134940129568|11.283478074825176|7.465888749197344|7.9678237014764575|7775.00150968661|105.01407407407405|0.4985378272134604|0.022930517693807093|2105.4396225071223|87.89925925925924| 2018-09-07 01:10:36|44.13|37.27|0.63|1.16|0.06|118.51| 57.48|between_snap|2555383.0|Jalen Mills| 31.0| CB| 15|home|2018090600| 190| left| null|7.0724889

In [0]:
#Choose only CB data
new_df2 = new_df.filter(new_df.position=="CB")
#Using SQL
new_df2.createOrReplaceTempView('new_df2')

In [0]:
results = spark.sql("SELECT gameId, playId, position, jerseyNumber, frameId, probability, prediction FROM new_df2 WHERE playId==75" )
results.show()

+----------+------+--------+------------+-------+--------------------+----------+
 gameId|playId|position|jerseyNumber|frameId| probability|prediction|
+----------+------+--------+------------+-------+--------------------+----------+
2018090600| 75| CB| 21.0| 36|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 37|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 38|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 39|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 40|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 41|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 42|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 43|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 44|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 45|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 46|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 47|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 48|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 49|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 50|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 51|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 52|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 53|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 54|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 55|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 56|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 57|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 58|[8.91644925230254...| 1|
2018090600| 75| CB| 21.0| 59|[8.91644925230254...| 1|
2018090600| 75| CB| 31.0| 12|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 13|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 14|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 15|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 16|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 17|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 18|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 19|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 20|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 21|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 22|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 23|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 24|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 25|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 26|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 27|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 28|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 29|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 30|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 31|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 32|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 33|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 34|[1.58690807062152...| 1|
2018090600| 75| CB| 31.0| 35|[1.58690807062152...| 1|
2018090600| 75| CB| 21.0| 1|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 2|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 3|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 4|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 5|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 6|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 7|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 8|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 9|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 10|[1.82180651341777...| 1|
2018090600| 75| CB| 21.0| 11|[1.82180651341777...| 1|
2018090600| 75| CB| 31.0| 36|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 37|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 38|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 39|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 40|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 41|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 42|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 43|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 44|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 45|[0.14268256007165...| 1|
2018090600| 75| CB| 31.0| 46|[0.14268256007165...